In [5]:
outputdirectory = "../docx-researchmap-outputs/"

file_name_save = outputdirectory+"filledintest.xlsx" #ダウンロードされるファイル名



In [26]:
import requests,json,sys,os, gspread, time, re, openpyxl
import numpy as np
if 'google.colab' in str(get_ipython()):
    %pip install python-docx
    from google.colab import files,auth
    from oauth2client.client import GoogleCredentials
    outputdirectory = ''
else:
    outputdirectory = '../docx-researchmap-outputs/' #ローカルで実行する場合は保存ファイルのディレクトリを適当に指定
    os.makedirs(outputdirectory,exist_ok=True)
import pandas as pd
file_name=outputdirectory+file_name_save

In [62]:
#スプレッドシートをダウンロード
sheeturl='https://docs.google.com/spreadsheets/d/1wce1XHSFGSBttupnSIqe_5abtijBb_hBYM2bfaV9Jn4/edit#gid=0' #作成したgoogle spreadsheetのアドレス
sheeturl_csv=re.match("https://docs.google.com/spreadsheets/d/.+/",sheeturl).group(0)+"export?format=csv"
name_data=pd.read_csv(sheeturl_csv)

,班,番号,代表分担協力,Surname,First name,苗字,名,researchmapID,grantID,Start date,End date,著者名（2個目）,著者名（3個目）
0,A,1,D,Okada,Yasushi,岡田,康志,yokadayokada,19H05795,2019-06-28,2024-03-31,NaN,NaN
1,A,1,B,Sasa,Shin-ichi,佐々,真一,sasa3341,19H05795,2019-06-28,2024-03-31,Sasa Shin-ishi,NaN
2,A,1,B,Kawaguchi,Kyogo,川口,喬吾,kyogok,19H05795,2019-06-28,2024-03-31,NaN,NaN
3,A,1,B,Kambara,Taketoshi,神原,丈敏,kambara,19H05795,2019-06-28,2024-03-31,NaN,NaN
4,A,1,B,Enoki,Sawako,榎,佐和子,senoki_research_1234,19H05795,2019-06-28,2024-03-31,NaN,NaN
5,A,1,B,Ikezaki,Keigo,池崎,圭吾,keigo_ikezaki,19H05795,2019-06-28,2024-03-31,NaN,NaN
6,B,1,D,Sagawa,Takahiro,沙川,貴大,taksagawa,19H05796,2019-06-28,2024-03-31,Sagawa T,NaN
7,B,1,B,Ito,Sosuke,伊藤,創祐,sosukeito,19H05796,2019-06-28,2024-03-31,NaN,NaN
8,B,1,K,Ashida,Keita,芦田,慶太,KAshida,19H05796,2019-06-28,2024-03-31,NaN,NaN
9,B,2,D,Ishijima,Akihiko,石島,秋彦,A.Ishijima,19H05797,2019-06-28,2024-03-31,NaN,NaN


In [57]:
#スプレッドシートをダウンロード
sheeturlform='https://docs.google.com/spreadsheets/d/1WrQwU1QKW5KnJX8ORxt1dihAH3M_AAv15HfIVCW9TrA/edit?resourcekey#gid=1794423468' #form由来のgoogle spreadsheetのアドレス
sheeturlform_csv=re.match("https://docs.google.com/spreadsheets/d/.+/",sheeturlform).group(0)+"export?format=csv"
col_names = ['c{0:02d}'.format(i) for i in range(46)]
name_dataform=pd.read_csv(sheeturlform_csv,header=0,names=col_names)
name_dataform=name_dataform.sort_values('c00', ascending=False)
name_dataform

,c00,c01,c02,c03,c04,c05,c06,c07,c08,c09,...,c36,c37,c38,c39,c40,c41,c42,c43,c44,c45
8,2021/05/13 19:12:39,dice-k@uec.ac.jp,中根 大介,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,大学を移動になり。単独運営の研究室所属となりました。
7,2021/05/13 18:43:19,mizuno@phys.kyushu-u.ac.jp,水野 大介,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,現在本領域に深く関係する研究分野の理論研究者向けの准教授ポストを、九大物理に新たに作るべく活...
6,2021/05/13 16:52:07,ariga@yamaguchi-u.ac.jp,有賀 隆行,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,九州大学・水野研、青山学院大学・富重研,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,私の所属している医学系研究科には修士課程がありませんが、医学部の学部生（学士入学なので修士号...
5,2021/05/13 16:06:47,soya.shinkai@riken.jp,新海 創也,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,遺伝学研究所・前島研、広島大学・落合研、医薬基盤研・今井研,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,特になしです。（ごめんなさい。）
4,2021/05/13 15:25:09,hosodakazufumi@gmail.com,細田 一史,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,東北大学・近藤倫生研、大阪大学・松田秀雄研、東京大学・古澤力研、水産研究教育機構・漁業情報解析室,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ロボット工学系の博士前期課程の学生３人をRAとして雇用した。うち1人は女性。学生からは、生命...
3,2021/05/13 14:48:13,kinjo@sci.hokudai.ac.jp,金城 政孝,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,東京医科歯科大学 認知行動医学講座,"Karolinska Institutet, Department of Clinical...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,関連研究にて学位取得1名
2,2021/05/13 13:43:53,tetsuya@sat.t.u-tokyo.ac.jp,小林 徹也,0.0,3.0,2.0,0.0,2.0,0.0,0.0,...,東京大学・若本研、理化学研究所・秋山研、慶応大学・舟橋研、近畿大学・山縣研,なし,なし,なし,American Academy of Sleep Medicine (AASM) Youn...,"C班共同会議; 化学走性の会議(非公開, 石島研、小林研、神野圭太 Yale Univers...",なし,なし,なし,なし
1,2021/05/12 18:30:07,amamori@fbs.osaka-u.ac.jp,石島 秋彦,2.0,1.0,NaN,NaN,NaN,NaN,2.0,...,東京大学・○○研、東北大学・▲▲研,ユタ大学・□□lab,理化学研究所・☆☆研,□□,日本物理学会 学生優秀発表賞（領域11）〇〇〇〇 2020年3月、日本物理学会 若手奨励賞（...,アクティブマター勉強会（岡田研、竹内研、川口研）、A班内共同会議）\n,〇〇〇〇（大学院生）、△△研、2020年1月に約2週間、〇〇実験手法の習得のため）,〇〇〇〇、2020年1月に約2週間、〇〇実験手法の習得のため）,〇〇〇〇、講演者、女子高生のためのサイエンスカフェ、2020年1月、 http://.......,川口先生、有難うございます！
0,2021/05/12 17:24:27,kyogo.kawaguchi@riken.jp,川口 喬吾,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,IMBA・RIVRON lab、Broad Insititute・Singh lab、Yal...,NaN,NaN,NaN,アクティブマター勉強会（岡田研、竹内研、川口研）、A班内共同会議（岡田研、佐々研、川口研）,NaN,NaN,理研BDRのHP改訂、ダイバーシティ・女性関連のページ作成https://www.bdr.r...,IPBセミナーにて、海外在住の若手研究者の発表の機会を作った（花井、武井）。


In [58]:
allnamesJP=(name_data["苗字"]+" "+name_data["名"]).to_list()
allnamesJP

timestamps=name_dataform.iloc[:,0].to_list()
namelist=name_dataform.iloc[:,2].to_list()
namelistKeikaku=[if ]


In [59]:
namelist

['中根 大介',
 '水野 大介',
 '有賀 隆行',
 '新海 創也',
 '細田 一史',
 '金城 政孝',
 '小林 徹也',
 '石島 秋彦',
 '川口 喬吾']

In [31]:
wb = openpyxl.load_workbook("./inputfiles/R3中間評価報告書（1_領域全体）（13参考データExcel版）.xlsx")
ws = wb.worksheets[0]


In [61]:
np.nansum(name_dataform.iloc[:,3].to_list())

2.0

In [36]:
# 研究代表者
## R1 計画
ws['C8'].value=8

## R1 公募
ws['D8'].value=0

## R2 計画
ws['E8'].value=8

## R2 公募
ws['F8'].value=8


#研究分担者
## R1 計画
ws['C9'].value=8

## R2 計画
ws['E8'].value=8

txt=[col+'{0:02d}'.format(i) for i in range(10,15) for col in ['C','D','E','F']]
print(txt)
#研究協力者


#若手研究者[1]


#外国人研究者


#ポスドク[2]


#RA等[2]


['C10', 'D10', 'E10', 'F10', 'C11', 'D11', 'E11', 'F11', 'C12', 'D12', 'E12', 'F12', 'C13', 'D13', 'E13', 'F13', 'C14', 'D14', 'E14', 'F14']


In [ ]:
wb.save(file_name)